In [1]:
import numpy as np
import cPickle
from collections import defaultdict
import sys, re
import pandas as pd

def build_data_cv(data_folder, cv=10, clean_string=True):
    """
    Loads data and split into 10 folds.
    """
    train_revs = []
    test_revs = []
    very_pos_file = data_folder[0]
    pos_file = data_folder[1]
    neutral_file = data_folder[2]
    neg_file = data_folder[3]
    very_neg_file = data_folder[4]
    test_file = data_folder[5]
    
    vocab = defaultdict(float)
    
    with open(very_pos_file, "rb") as f:
        for line in f:       
            rev = []
            rev.append(line.strip())
            if clean_string:
                orig_rev = clean_str(" ".join(rev))
            else:
                orig_rev = " ".join(rev).lower()
            words = set(orig_rev.split())
            for word in words:
                vocab[word] += 1
            datum  = {"y":4, 
                      "text": orig_rev,                             
                      "num_words": len(orig_rev.split()),
                      "split": np.random.randint(0,cv)}
            train_revs.append(datum)
    
    
    with open(pos_file, "rb") as f:
        for line in f:       
            rev = []
            rev.append(line.strip())
            if clean_string:
                orig_rev = clean_str(" ".join(rev))
            else:
                orig_rev = " ".join(rev).lower()
            words = set(orig_rev.split())
            for word in words:
                vocab[word] += 1
            datum  = {"y":3, 
                      "text": orig_rev,                             
                      "num_words": len(orig_rev.split()),
                      "split": np.random.randint(0,cv)}
            train_revs.append(datum)
            
    with open(neutral_file, "rb") as f:
        for line in f:       
            rev = []
            rev.append(line.strip())
            if clean_string:
                orig_rev = clean_str(" ".join(rev))
            else:
                orig_rev = " ".join(rev).lower()
            words = set(orig_rev.split())
            for word in words:
                vocab[word] += 1
            datum  = {"y":2, 
                      "text": orig_rev,                             
                      "num_words": len(orig_rev.split()),
                      "split": np.random.randint(0,cv)}
            train_revs.append(datum)
    
    with open(neg_file, "rb") as f:
        for line in f:       
            rev = []
            rev.append(line.strip())
            if clean_string:
                orig_rev = clean_str(" ".join(rev))
            else:
                orig_rev = " ".join(rev).lower()
            words = set(orig_rev.split())
            for word in words:
                vocab[word] += 1
            datum  = {"y":1, 
                      "text": orig_rev,                             
                      "num_words": len(orig_rev.split()),
                      "split": np.random.randint(0,cv)}
            train_revs.append(datum)
            
    with open(very_neg_file, "rb") as f:
        for line in f:       
            rev = []
            rev.append(line.strip())
            if clean_string:
                orig_rev = clean_str(" ".join(rev))
            else:
                orig_rev = " ".join(rev).lower()
            words = set(orig_rev.split())
            for word in words:
                vocab[word] += 1
            datum  = {"y":0, 
                      "text": orig_rev,                             
                      "num_words": len(orig_rev.split()),
                      "split": np.random.randint(0,cv)}
            train_revs.append(datum)
            
    with open(test_file, "rb") as f:
        for line in f:       
            rev = []
            rev.append(line.strip())
            if clean_string:
                orig_rev = clean_str(" ".join(rev))
            else:
                orig_rev = " ".join(rev).lower()
            words = set(orig_rev.split())
            for word in words:
                vocab[word] += 1
            datum  = {"text": orig_rev,                             
                      "num_words": len(orig_rev.split())}
            test_revs.append(datum)
            
    return train_revs, test_revs, vocab
    
def get_W(word_vecs, k=300):
    """
    Get word matrix. W[i] is the vector for word indexed by i
    """
    vocab_size = len(word_vecs)
    word_idx_map = dict()
    W = np.zeros(shape=(vocab_size+1, k), dtype='float32')            
    W[0] = np.zeros(k, dtype='float32')
    i = 1
    for word in word_vecs:
        W[i] = word_vecs[word]
        word_idx_map[word] = i
        i += 1
    return W, word_idx_map

def load_bin_vec(fname, vocab):
    """
    Loads 300x1 word vecs from Google (Mikolov) word2vec
    """
    word_vecs = {}
    with open(fname, "rb") as f:
        header = f.readline()
        vocab_size, layer1_size = map(int, header.split())
        binary_len = np.dtype('float32').itemsize * layer1_size
        for line in xrange(vocab_size):
            word = []
            while True:
                ch = f.read(1)
                if ch == ' ':
                    word = ''.join(word)
                    break
                if ch != '\n':
                    word.append(ch)   
            if word in vocab:
                word_vecs[word] = np.fromstring(f.read(binary_len), dtype='float32')  
            else:
                f.read(binary_len)
    return word_vecs

def add_unknown_words(word_vecs, vocab, min_df=1, k=300):
    """
    For words that occur in at least min_df documents, create a separate word vector.    
    0.25 is chosen so the unknown vectors have (approximately) same variance as pre-trained ones
    """
    for word in vocab:
        if word not in word_vecs and vocab[word] >= min_df:
            word_vecs[word] = np.random.uniform(-0.25,0.25,k)  

def clean_str(string, TREC=False):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Every dataset is lower cased except for TREC
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)     
    string = re.sub(r"\'s", " \'s", string) 
    string = re.sub(r"\'ve", " \'ve", string) 
    string = re.sub(r"n\'t", " n\'t", string) 
    string = re.sub(r"\'re", " \'re", string) 
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r",", " , ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " \( ", string) 
    string = re.sub(r"\)", " \) ", string) 
    string = re.sub(r"\?", " \? ", string) 
    string = re.sub(r"\s{2,}", " ", string)    
    return string.strip() if TREC else string.strip().lower()

def clean_str_sst(string):
    """
    Tokenization/string cleaning for the SST dataset
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)   
    string = re.sub(r"\s{2,}", " ", string)    
    return string.strip().lower()

def convert_keys_to_string(dictionary):
    """Recursively converts dictionary keys to strings."""
    if not isinstance(dictionary, dict):
        return dictionary
    return dict((str(k), convert_keys_to_string(v)) 
        for k, v in dictionary.items())

In [2]:
data_folder = ["rt-polarity.verypos","rt-polarity.pos","rt-polarity.neutral","rt-polarity.neg",
               "rt-polarity.veryneg", "test.data"]    
print "Loading data...",        
train_revs, test_revs, vocab = build_data_cv(data_folder, cv=10, clean_string=True)
max_l_train = np.max(pd.DataFrame(train_revs)["num_words"])
max_l_test = np.max(pd.DataFrame(test_revs)["num_words"])
print "Data Loaded!"
print "Number of train sentences: " + str(len(train_revs))
print "Number of test sentences: " + str(len(test_revs))
print "Vocab size: " + str(len(vocab))
print "Max train sentence length: " + str(max_l_train)
print "Max test sentence length: " + str(max_l_test)
vocab = convert_keys_to_string(vocab)

Loading data... Data Loaded!
Number of train sentences: 10564
Number of test sentences: 100
Vocab size: 18765
Max train sentence length: 56
Max test sentence length: 41


In [72]:
import gensim
model = gensim.models.Word2Vec.load('C:\Scisoft\My Current Projects\Neural Networks All\Word Embeddings\glove_6B_300d')
glove = {}
for item in vocab.keys():
    try:
        glove[item] = model[item] 
    except KeyError:
        continue
print 'GloVe model loaded.'

w2v_file = 'C:\Scisoft\My Current Projects\Neural Networks All\Word Embeddings\GoogleNews-vectors-negative300.bin'  
w2v = load_bin_vec(w2v_file, vocab)
print 'Word2Vec model loaded.'

GloVe model loaded.
Word2Vec model loaded.


In [73]:
print "Num words already in Word2Vec: " + str(len(w2v))
add_unknown_words(w2v, vocab)
W1, word_idx_map = get_W(w2v)

print "Num words already in glove: " + str(len(glove))
add_unknown_words(glove, vocab)
W2, _ = get_W(glove)

W3, _ = get_W(rand_vecs)

cPickle.dump([train_revs, test_revs, W1, W2, W3, word_idx_map, vocab], open("mr.p", "wb"))
print "Dataset created with Word2Vec, GloVe and Random word vectors"

Num words already in Word2Vec: 16448
Num words already in glove: 17609
Dataset created with Word2Vec, GloVe and Random word vectors


In [76]:
x = cPickle.load(open("mr.p","rb"))
train_revs, test_revs, W1, W2, W3, word_idx_map, vocab = x[0], x[1], x[2], x[3], x[4], x[5], x[6]